In [41]:
import pandas as pd
import os
%config Completer.use_jedi = False
csvpath = os.path.join("HeroesOfPymoli","Resources","purchase_data.csv")

purchase_data = pd.read_csv(csvpath)

pymoli_df = pd.DataFrame(purchase_data)

pymoli_df.head()


,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


In [42]:
player_count_df = pymoli_df["SN"].nunique()
playerdict = [{"Total Players" : (player_count_df)}]
totalplayers = pd.DataFrame(playerdict)
totalplayers

,Total Players
0,576


In [43]:
#Count of unique items
item_types_df = pymoli_df["Item ID"].nunique()

#Calculate average purchase price
ave_price = pymoli_df["Price"].mean()
ave_price_currency = "${:.2f}".format(ave_price)
#Total number of purchases
num_purchases = pymoli_df["Purchase ID"].count()

#calculate total Revenue
total_revenue = pymoli_df["Price"].sum()
total_revenue_currency = "${:.2f}".format(total_revenue)
#create dictionary
purchasing_analysis_dict = [{"Number of Unique Items" : (item_types_df), "Average Price" : (ave_price_currency), "Number of Purchases": (num_purchases), "Total Revenue" : (total_revenue_currency)}]

purchasing_analysis_df = pd.DataFrame(purchasing_analysis_dict)

purchasing_analysis_df


,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,179,$3.05,780,$2379.77


In [44]:
#gender demos and total percentage
player_count = len(purchase_data["SN"].unique())

#players
male_players = purchase_data.loc[purchase_data["Gender"] == "Male"]
male_count = len(male_players["SN"].unique())
male_percent = "{:.2f}%".format(male_count / player_count * 100)

female_players = purchase_data.loc[purchase_data["Gender"] == "Female"]
female_count = len(female_players["SN"].unique())
female_percent = "{:.2f}%".format(female_count / player_count * 100)

other_players = purchase_data.loc[purchase_data["Gender"] == "Other / Non-Disclosed"]
other_count = len(other_players["SN"].unique())
other_percent = "{:.2f}%".format(other_count / player_count * 100)

#summary
gender_demographics_table = pd.DataFrame([{
    "Gender": "Male", "Total Count": male_count, 
    "Percentage of Players": male_percent}, 
    {"Gender": "Female", "Total Count": female_count, 
     "Percentage of Players": female_percent}, 
    {"Gender": "Other / Non-Disclosed", "Total Count": other_count, 
     "Percentage of Players": other_percent
    }], columns=["Gender", "Total Count", "Percentage of Players"])

gender_demographics_table = gender_demographics_table.set_index("Gender")
gender_demographics_table.index.name = "Gender"
gender_demographics_table



,Total Count,Percentage of Players
Gender,,
Male,484,84.03%
Female,81,14.06%
Other / Non-Disclosed,11,1.91%


In [45]:
#create grouped data frame
gender_grouped_df = pymoli_df.copy()
gender_grouped_df = pymoli_df.groupby(["Gender"])
gpurchase_count = gender_grouped_df["Purchase ID"].count()
gprice_ave = gender_grouped_df["Price"].mean().map("${:.2f}".format)
gtotal_value =gender_grouped_df["Price"].sum().map("${:.2f}".format)

gendertotals = gender_grouped_df["Price"].sum()
individuals = gender_grouped_df["SN"].nunique()
ave_per_player = (gendertotals/individuals).map("${:.2f}".format)
#combine columns for display
gender_analysis_df = pd.DataFrame({"Purchase Count" : gpurchase_count, "Average Purchase Price" : gprice_ave, "Total Purchase Value" : gtotal_value, "Avg Total Purchase Per Person" : ave_per_player})

gender_analysis_df





,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase Per Person
Gender,,,,
Female,113,$3.20,$361.94,$4.47
Male,652,$3.02,$1967.64,$4.07
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


In [46]:
#create a copy data frame to work with
player_age_df = pymoli_df.copy()
#create bins
bins = [0, 9, 14, 19, 24, 29, 34, 39, 100]
binlabels = ["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"]
age_groups = pd.cut(player_age_df["Age"], bins, labels = binlabels)
player_age_df["Age Bracket"] = age_groups
player_age_df.drop_duplicates(["SN"], inplace=True)

player_ages = player_age_df["Age Bracket"].value_counts()

percents = player_age_df["Age Bracket"].value_counts(normalize=True)*100
percent_age = (percents).map("{:.2f}%".format)
#combine and sort age and percentage for display
age_df = pd.DataFrame({"Total Count" : player_ages, "Percentage of Players" : percent_age})
age_analysis = age_df.sort_index()
age_analysis

,Total Count,Percentage of Players
<10,17,2.95%
10-14,22,3.82%
15-19,107,18.58%
20-24,258,44.79%
25-29,77,13.37%
30-34,52,9.03%
35-39,31,5.38%
40+,12,2.08%


In [47]:
#purchasing analysis age
spender_age_df = pymoli_df.copy()
spender_age_df.head()
bins = [0, 9, 14, 19, 24, 29, 34, 39, 100]
binlabels = ["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"]
spender_age_df["Age Bracket"] = pd.cut(spender_age_df["Age"], bins, labels = binlabels)
spender_age_df
#create data

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price,Age Bracket
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53,20-24
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56,40+
2,2,Ithergue48,24,Male,92,Final Critic,4.88,20-24
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27,20-24
4,4,Iskosia90,23,Male,131,Fury,1.44,20-24
...,...,...,...,...,...,...,...,...
775,775,Aethedru70,21,Female,60,Wolf,3.54,20-24
776,776,Iral74,21,Male,164,Exiled Doomblade,1.63,20-24
777,777,Yathecal72,20,Male,67,"Celeste, Incarnation of the Corrupted",3.46,20-24
778,778,Sisur91,7,Male,92,Final Critic,4.19,<10


In [48]:
spender_age_df.drop_duplicates(["SN"], inplace=True)
total_purchase = spender_age_df.groupby(["Age Bracket"]).sum()["Price"].rename("Total Purchase Value")

average_purchase = spender_age_df.groupby(["Age Bracket"]).mean()["Price"]
count_purchase = spender_age_df.groupby(["Age Bracket"]).count()["Price"].rename("Purchase Count")

purchase_df = pd.DataFrame({"Total Purchase Value" : total_purchase,
                          "Average Purchase Price" : average_purchase,
 "Purchase Count" : count_purchase })

purchase_df

,Total Purchase Value,Average Purchase Price,Purchase Count
Age Bracket,,,
<10,57.63,3.390000,17
10-14,67.64,3.074545,22
15-19,331.88,3.101682,107
20-24,790.39,3.063527,258
25-29,223.93,2.908182,77
30-34,151.92,2.921538,52
35-39,108.81,3.510000,31
40+,36.45,3.037500,12


In [53]:
#top spenders
spender_df = pymoli_df.copy()

total_purchase = spender_df.groupby(["SN"]).sum()["Price"].rename("Total Purchase Value")

average_purchase = spender_df.groupby(["SN"]).mean()["Price"]
count_purchase = spender_df.groupby(["SN"]).count()["Price"].rename("Purchase Count")
spender_df = pd.DataFrame({"Total Purchase Value" : total_purchase,
                          "Average Purchase Price" : average_purchase,
 "Purchase Count" : count_purchase })
count_purchase

#formal data
spender_df_final = spender_df.sort_values(by = ["Total Purchase Value"] , ascending = [False])
spender_df_final["Average Purchase Price"] = spender_df_final["Average Purchase Price"].map("${:.2f}".format)
spender_df_final["Total Purchase Value"] = spender_df_final["Total Purchase Value"].map("${:.2f}".format)

spender_df_final.head()





,Total Purchase Value,Average Purchase Price,Purchase Count
SN,,,
Lisosia93,$18.96,$3.79,5
Idastidru52,$15.45,$3.86,4
Chamjask73,$13.83,$4.61,3
Iral74,$13.62,$3.40,4
Iskadarya95,$13.10,$4.37,3


In [ ]:
popular_df = pymoli_df.copy()
popular_df = popular_df.drop("Purchase ID", "SN", "Age","Gender"axis =1)
item_id_var = popular_df.groupby(["Item ID"]).count()["Price"].rename("Purchase Count") 
popular_count = popular_df.groupby(["Item ID"]).count()["Price"].rename("Purchase Count")
populat_total_value = popular_df.groupby(["Item ID"]).sum()["Price"].rename("Total Purchase Value")